# **Introduction**


This project analyzes the 2016 Rio Olympic Games using SQL for data extraction and processing and Tableau for data visualization. The dataset, sourced from https://www.kaggle.com/datasets/rio2016/olympic-games, contains information on athletes, countries, and events. The analysis focuses on three key areas:

* **Athlete Demographics:** Comparing age, height, and weight across sports and genders.
* **Event Distribution:** Visualizing the concentration of events in Rio’s neighborhoods and the distribution of events across sports and disciplines.
* **Country Performance:** Ranking the top-performing countries by gold medals, adjusted for population, GDP, and overall metrics.

The visualizations can be viewed in: [https://public.tableau.com/app/profile/idni.tav/vizzes].



# Data Import and Setup

In [1]:
#importing libraries
import numpy as np 
import pandas as pd 
import sqlite3 #for working with SQL databases

#setting pandas to display full data 
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', None) 

#kaggle's default code block to import data 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/olympic-games/countries.csv
/kaggle/input/olympic-games/athletes.csv
/kaggle/input/olympic-games/events.csv


In [2]:
# Loading and storing data from olympic-games
#'ISO-8859-1' encoding for special characters 
countries=pd.read_csv('/kaggle/input/olympic-games/countries.csv', encoding='ISO-8859-1')
athletes=pd.read_csv('/kaggle/input/olympic-games/athletes.csv', encoding='ISO-8859-1')
events=pd.read_csv('/kaggle/input/olympic-games/events.csv', encoding='ISO-8859-1')

In [3]:
#creating temporary database in memory to use SQL
connection = sqlite3.connect(':memory:')

#loading data into the database
countries.to_sql('countries', connection, index=False, if_exists='replace')
athletes.to_sql('athletes', connection, index=False, if_exists='replace')
events.to_sql('events', connection, index=False, if_exists='replace')


306

In [4]:
#preview of country data 
countries.head()

,country,code,population,gdp_per_capita
0,Afghanistan,AFG,32526562.0,594.323081
1,Albania,ALB,2889167.0,3945.217582
2,Algeria,ALG,39666519.0,4206.031232
3,American Samoa*,ASA,55538.0,NaN
4,Andorra,AND,70473.0,NaN


In [5]:
#preview of athletes data 
athletes.head()

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze
0,736041664,A Jesus Garcia,ESP,male,10/17/69,1.72,64.0,athletics,0,0,0
1,532037425,A Lam Shin,KOR,female,9/23/86,1.68,56.0,fencing,0,0,0
2,435962603,Aaron Brown,CAN,male,5/27/92,1.98,79.0,athletics,0,0,1
3,521041435,Aaron Cook,MDA,male,1/2/91,1.83,80.0,taekwondo,0,0,0
4,33922579,Aaron Gate,NZL,male,11/26/90,1.81,71.0,cycling,0,0,0


In [6]:
#preview of events data
events.head()

,id,sport,discipline,name,sex,venues
0,701492,aquatics,backstroke,Women's 100m Backstroke,female,Olympic Aquatics Stadium
1,305278,aquatics,backstroke,Women's 200m Backstroke,female,Olympic Aquatics Stadium
2,708010,aquatics,backstroke,Men's 100m Backstroke,male,Olympic Aquatics Stadium
3,729643,aquatics,backstroke,Men's 200m Backstroke,male,Olympic Aquatics Stadium
4,567019,aquatics,breaststroke,Women's 100m Breaststroke,female,Olympic Aquatics Stadium


# Athlete Demographics + Data Cleaning

**Visualizations:** 
* 3 Boxplot Charts

*athletes_stats_query* aims to extract information on the athlete's age, height, and weight, to be displayed on boxplots for each sport and gender. While all relevant data is in the *athletes* table, the date of birth (dob) needs to be converted into age.

In [7]:
athletes.dob.head()

0    10/17/69
1     9/23/86
2     5/27/92
3      1/2/91
4    11/26/90
Name: dob, dtype: object

Since the date formats are inconsistent, the first step is to standardize them by adding leading zeros to fit the format *mm/dd/yy*. This is done in the first CTE, *zerod_dates*.

The next step is to convert the dates into SQLite's default format, *yyyy-mm-dd*, which is done in the second CTE, *formatted_dates*. This conversion includes a CASE condition to account for whether the year is before or after 2000.

The athlete's age is calculated using the newly formatted date of birth and the start date of the Olympic Games, *2016-08-05*.

In [8]:
athletes_stats_query="""

             --STEP 1:add zeros
             WITH zerod_dates AS(
                                    SELECT *,printf('%02d/%02d/%02d',
                                        CAST(substr(dob, 1, instr(dob, '/') - 1) AS INTEGER),
                                        CAST(substr(dob,instr(dob,'/')+1,instr(substr(dob,instr(dob,'/')+1),'/')-1) AS INTEGER), 
                                        CAST(substr(substr(dob,instr(dob,'/')+1),instr(substr(dob,instr(dob,'/')+1),'/')+1) AS INTEGER)) AS zerod_dob
                                    FROM athletes 
             ),
             
             --STEP 2: change date format to yyyy-mm-dd
             formatted_dates AS(
                                    SELECT *,
                                        CASE WHEN CAST(substr(zerod_dob, 7, 2) AS INTEGER)<=16 
                                        THEN printf('20%02d-%02d-%02d',
                                            CAST(substr(zerod_dob, 7, 2) AS INTEGER),
                                            CAST(substr(zerod_dob, 1, 2) AS INTEGER),
                                            CAST(substr(zerod_dob, 4, 2) AS INTEGER))
                                        ELSE printf('19%02d-%02d-%02d',
                                            CAST(substr(zerod_dob, 7, 2) AS INTEGER),
                                            CAST(substr(zerod_dob, 1, 2) AS INTEGER),
                                            CAST(substr(zerod_dob, 4, 2) AS INTEGER)) 
                                        END AS formatted_dob
                                    FROM zerod_dates
             )
             
             --STEP 3: extract relevant columns + compute age 
             SELECT sport,sex, height, weight,strftime("%Y","2016-08-05") - strftime("%Y",formatted_dob) 
             - (CASE WHEN strftime('%m-%d', '2016-08-05') < strftime('%m-%d',formatted_dob) THEN 1 ELSE 0 
             END) AS age
             FROM formatted_dates
             ORDER BY sport, sex
        
"""
#saving and printing dataframe
athletes_stats_df = pd.read_sql(athletes_stats_query, connection)
athletes_stats_df.head()


,sport,sex,height,weight,age
0,aquatics,female,1.78,68.0,19.0
1,aquatics,female,1.66,61.0,26.0
2,aquatics,female,1.58,53.0,17.0
3,aquatics,female,1.70,58.0,18.0
4,aquatics,female,1.75,60.0,22.0


Syntax for future reference: 

In [9]:
#printf: formats strings or numbers according to the specified format
#syntax: printf(format_string, value_1, value_2,...)
#example: printf('%02d', 5) = '05'

#substr: extracts a portion of a string
#syntax: substr(string, start_position, length)
#example: substr('abcd', 2, 2) = 'bc'

#instr: finds the position of the first time a specific portion of a string appears on the original string
#syntax: instr(string, substring)
#example: instr('hello world', 'world') = 7

# Event Distribution 

**Visualizations:** 
* 1 Treemap
* 1 Event Density Map


*sports_events_query* extracts data from *events* and counts the number of occurences in each sport and discipline. The outputed table is used on a treemap plot to visualize the prevalence of different sports on the Olympic Games of that year.

In [10]:
sport_events_query="""
                   SELECT sport,discipline,COUNT(id) AS event_count
                   FROM events
                   GROUP BY sport, discipline
"""
#saving and printing dataframe
sport_events_df = pd.read_sql(sport_events_query, connection)
sport_events_df.head()

,sport,discipline,event_count
0,aquatics,backstroke,4
1,aquatics,breaststroke,4
2,aquatics,butterfly,4
3,aquatics,diving,8
4,aquatics,freestyle,14


To visualize the geographical distribution of events across Rio de Janeiro, the coordinates of the venues where each event takes place must be added to the project. The *venue_coordinates* table is first created in the database. Then, the coordinates for each venue are manually inserted into the table. Finally, *venue_coordinates* is joined with *events* on *venues_query*.

In [11]:
#creating a table in the database
connection.execute("""
                   CREATE TABLE IF NOT EXISTS venue_coordinates (
                   venue TEXT PRIMARY KEY,
                   latitude REAL,
                   longitude REAL
                    )
""")

In [12]:
#checking the venues
events.venues.unique()

array(['Olympic Aquatics Stadium', 'Maria Lenk Aquatics Centre',
       'Fort Copacabana',
       'Maria Lenk Aquatics Centre, Olympic Aquatics Stadium',
       'SambÌ_dromo', 'Olympic Stadium', 'Pontal',
       'Riocentro - Pavilion 4', 'Carioca Arena 1, Youth Arena',
       'Carioca Arena 1', 'Riocentro - Pavilion 6', 'Whitewater Stadium',
       'Lagoa Stadium', 'Olympic BMX Centre', 'Mountain Bike Centre',
       'Rio Olympic Velodrome', 'Olympic Equestrian Centre',
       'Carioca Arena 3',
       'MineirÌ£o, Olympic Stadium, Fonte Nova Arena, AmazÌ«nia Arena, Corinthians Arena, ManÌ© Garrincha Stadium, MaracanÌ£',
       'Olympic Golf Course', 'Rio Olympic Arena', 'Future Arena',
       'Olympic Hockey Centre', 'Carioca Arena 2',
       'Deodoro Aquatics Centre, Deodoro Stadium, Youth Arena',
       'Deodoro Stadium', 'Marina da GlÌ_ria', 'Olympic Shooting Centre',
       'Riocentro - Pavilion 3', 'Olympic Tennis Centre',
       'Beach Volleyball Arena', 'MaracanÌ£zinho',
       

In [13]:
#adding coordinates to the table
#for simplicity the 3 events held in multiple venues were assigned the coordinates of the 
#first listed venue, which may lead to slight innacuracies
connection.execute("""
INSERT OR REPLACE INTO venue_coordinates(venue, latitude, longitude) 
VALUES 
('Olympic Aquatics Stadium', -22.9807, -43.3949),
('Maria Lenk Aquatics Centre', -22.9744, -43.3878),
('Maria Lenk Aquatics Centre, Olympic Aquatics Stadium', -22.9807, -43.3949),
('SambÌ_dromo', -22.8953, -43.1964),
('Olympic Stadium', -22.8932, -43.2923),
('Pontal', -23.0305, -43.4737),
('Riocentro - Pavilion 4', -22.9799, -43.4116),
('Carioca Arena 1, Youth Arena', -22.8594, -43.4042),
('Carioca Arena 1', -22.9765, -43.3923),
('Riocentro - Pavilion 6', -22.9799, -43.4116),
('Whitewater Stadium', -22.8486, -43.4026),
('Lagoa Stadium', -22.9711, -43.2117),
('Olympic BMX Centre', -22.8466, -43.4011),
('Mountain Bike Centre', -22.8583, -43.4125),
('Rio Olympic Velodrome', -22.9747, -43.3922),
('Olympic Equestrian Centre', -22.8694, -43.4058),
('Carioca Arena 3', -22.9766, -43.3924),
('MineirÌ£o, Olympic Stadium, Fonte Nova Arena, AmazÌ«nia Arena, Corinthians Arena, ManÌ© Garrincha Stadium, MaracanÌ£', -22.9122, -43.2303),
('Olympic Golf Course', -23.0050, -43.4030),
('Rio Olympic Arena', -22.9753, -43.3903),
('Future Arena', -22.9805, -43.3922),
('Olympic Hockey Centre', -22.8607, -43.4024),
('Carioca Arena 2', -22.9777, -43.3924),
('Deodoro Aquatics Centre, Deodoro Stadium, Youth Arena', -22.8618, -43.4036),
('Deodoro Stadium', -22.8609, -43.4066),
('Marina da GlÌ_ria', -22.9203, -43.1697),
('Olympic Shooting Centre', -22.8589, -43.4112),
('Riocentro - Pavilion 3', -22.9773, -43.4126),
('Olympic Tennis Centre', -22.9788, -43.3963),
('Beach Volleyball Arena', -22.9658, -43.1733),
('MaracanÌ£zinho', -22.9140, -43.2293),
('Riocentro - Pavilion 2', -22.9761, -43.4114)

""")

In [14]:
#joining tables
venues_query="""
                   SELECT e.venues, e.name, e.discipline, e.sport, v.latitude, v.longitude
                   FROM events AS e
                   JOIN venue_coordinates AS v 
                       ON e.venues = v.venue
"""

#saving and printing dataframe
venues_df = pd.read_sql(venues_query, connection)
venues_df.head()


,venues,name,discipline,sport,latitude,longitude
0,Olympic Aquatics Stadium,Women's 100m Backstroke,backstroke,aquatics,-22.9807,-43.3949
1,Olympic Aquatics Stadium,Women's 200m Backstroke,backstroke,aquatics,-22.9807,-43.3949
2,Olympic Aquatics Stadium,Men's 100m Backstroke,backstroke,aquatics,-22.9807,-43.3949
3,Olympic Aquatics Stadium,Men's 200m Backstroke,backstroke,aquatics,-22.9807,-43.3949
4,Olympic Aquatics Stadium,Women's 100m Breaststroke,breaststroke,aquatics,-22.9807,-43.3949


# Country Performance

**Visualizations:** 
* 3 Bar Charts
* 1 Sunburts Plot + Map

Determining the best countries at the Olympic can be subjective due to the diverse metrics and unique characteristics of each nation, such as size, population, sports culture, and economic prosperity. For simplicty, this analysis focuses only on gold medals, specifically in the following aspects: total number of medals,number of medals adjusted for population, and number of medals adjusted for economic power.







To ensure meaningful results and minimize the impact of random factors, the queries focus on the top 15 countries in each category, considering only those that have won at least 3 gold medals.

*gold_total_query* extracts data from *athletes* to retrieve the top 15 countries with the highest total number of gold medals.

In [17]:
gold_total_query="""
                 SELECT nationality AS country_code, COUNT(*) AS gold_medal_count
                 FROM athletes
                 WHERE gold=1
                 GROUP BY nationality
                 HAVING COUNT(CASE WHEN gold = 1 THEN 1 ELSE NULL END) >= 3
               
                 ORDER BY gold_medal_count DESC
                 LIMIT 15
"""
#saving and printing dataframe
gold_total_df = pd.read_sql(gold_total_query, connection)
print(gold_total_df)

   country_code  gold_medal_count
0           USA               107
1           GBR                55
2           RUS                46
3           GER                43
4           CHN                38
5           BRA                37
6           AUS                23
7           ARG                21
8           FRA                20
9           JPN                15
10          DEN                15
11          SRB                14
12          FIJ                13
13          NED                 9
14          KOR                 9


*medal_per_million_query* incorporates population data by joining *countries* to *athletes*. This enables the calculation of the number of gold medals per million inhabitants, offering a clearer measure of success relative to population size. To avoid skewed results from very small countries, only nations with over a million people were considered.

In [15]:
medal_per_million_query="""
                        SELECT a.nationality AS country_code,ROUND(SUM(CASE WHEN a.gold=1 THEN 1 ELSE 0 END)/c.population*1000000,2) AS gold_per_million
                        FROM athletes AS a
                            LEFT JOIN countries AS c
                            ON a.nationality=c.code
                        WHERE  c.population>=1000000 AND a.gold=1
                        GROUP BY a.nationality
                        HAVING COUNT(CASE WHEN a.gold = 1 THEN 1 ELSE NULL END) >= 3
                        ORDER BY gold_per_million DESC
                        LIMIT 15
                        
"""
#saving and printing dataframe
medals_per_million_df = pd.read_sql(medal_per_million_query, connection)
print(medals_per_million_df)

   country_code  gold_per_million
0           DEN              2.64
1           JAM              2.20
2           CRO              1.66
3           NZL              1.31
4           AUS              0.97
5           GBR              0.84
6           SUI              0.72
7           SVK              0.55
8           NED              0.53
9           GER              0.53
10          ARG              0.48
11          CUB              0.44
12          HUN              0.41
13          USA              0.33
14          RUS              0.32


*medal_per_10billion_query* incorporates GDP data by joining *countries* to *athletes*. This enables the calculation of the number of gold medals per 10 billion of GDP (dollars), providing a clearer measure of success relative to economic power. To avoid skewed results from very small economies, only nations with a GDP of at least 20 billion dollars were considered.

In [16]:
medal_per_10billion_query="""
                        SELECT a.nationality AS country_code,ROUND(SUM(CASE WHEN a.gold=1 THEN 1 ELSE 0 END)/(c.population*c.gdp_per_capita)*10000000000,2) AS gold_per_10billion
                        FROM athletes AS a
                            LEFT JOIN countries AS c
                            ON a.nationality=c.code
                        WHERE  c.gdp_per_capita*c.population>=20000000000 AND a.gold=1
                        
                        GROUP BY a.nationality
                        HAVING COUNT(CASE WHEN a.gold = 1 THEN 1 ELSE NULL END) >= 3
                        ORDER BY gold_per_10billion DESC
                        LIMIT 15
                        
"""
#saving and printing dataframe
medals_per_10billion_df = pd.read_sql(medal_per_10billion_query, connection)
print(medals_per_10billion_df)

   country_code  gold_per_10billion
0           CRO                1.44
1           KEN                0.95
2           UZB                0.60
3           DEN                0.51
4           ARG                0.36
5           RUS                0.35
6           NZL                0.35
7           SVK                0.34
8           HUN                0.33
9           BRA                0.21
10          GBR                0.19
11          AUS                0.17
12          KAZ                0.16
13          GRE                0.15
14          GER                0.13


The next query combines the results of the three previous queries using a FULL JOIN. It is a redundant step, serving  the sole purpose of reformating the data to facilitate editing on Tableau and create a sunburst plot.

In [18]:
sunburst_plot_query = """
--gold_total_query
WITH gold_total AS (
    SELECT nationality AS total_code
    FROM athletes
    WHERE gold = 1
    GROUP BY nationality
    HAVING COUNT(CASE WHEN gold = 1 THEN 1 ELSE NULL END) >= 3
    ORDER BY COUNT(*) DESC
    LIMIT 15
),

--medal_per_million_query
gold_per_million AS (
    SELECT a.nationality AS population_code
    FROM athletes AS a
    LEFT JOIN countries AS c ON a.nationality = c.code
    WHERE c.population >= 1000000 AND a.gold = 1
    GROUP BY a.nationality
    HAVING COUNT(CASE WHEN a.gold = 1 THEN 1 ELSE NULL END) >= 3
    ORDER BY SUM(CASE WHEN a.gold = 1 THEN 1 ELSE 0 END) / c.population * 1000000 DESC
    LIMIT 15
),
--medal_per_billion_query
gold_per_10billion AS (
    SELECT a.nationality AS gdp_code
    FROM athletes AS a
    LEFT JOIN countries AS c ON a.nationality = c.code
    WHERE c.gdp_per_capita * c.population >= 20000000000 AND a.gold = 1
    GROUP BY a.nationality
    HAVING COUNT(CASE WHEN a.gold = 1 THEN 1 ELSE NULL END) >= 3
    ORDER BY SUM(CASE WHEN a.gold = 1 THEN 1 ELSE 0 END) / (c.population * c.gdp_per_capita) * 10000000000 DESC
    LIMIT 15
)
--FULL JOIN
SELECT t.total_code AS "Total Code",
       p.population_code AS "Population Code",
       g.gdp_code AS "GDP Code"
FROM gold_total t
FULL OUTER JOIN gold_per_million p ON t.total_code = p.population_code
FULL OUTER JOIN gold_per_10billion g ON COALESCE(t.total_code, p.population_code) = g.gdp_code
"""

#saving and printing dataframe
sunburst_plot_df = pd.read_sql(sunburst_plot_query, connection)
print(sunburst_plot_df)

   Total Code Population Code GDP Code
0         USA             USA     None
1         GBR             GBR      GBR
2         RUS             RUS      RUS
3         GER             GER      GER
4         CHN            None     None
5         BRA            None      BRA
6         AUS             AUS      AUS
7         ARG             ARG      ARG
8         FRA            None     None
9         JPN            None     None
10        DEN             DEN      DEN
11        SRB            None     None
12        FIJ            None     None
13        NED             NED     None
14        KOR            None     None
15       None             JAM     None
16       None             CRO      CRO
17       None             NZL      NZL
18       None             SUI     None
19       None             SVK      SVK
20       None             CUB     None
21       None             HUN      HUN
22       None            None      KEN
23       None            None      UZB
24       None            

# Saving Output Data

In [19]:
sport_events_df.to_csv('/kaggle/working/sports_events',index=False)
medals_per_million_df.to_csv('/kaggle/working/medals_per_million',index=False)
medals_per_10billion_df.to_csv('/kaggle/working/medals_per_10billion',index=False)
gold_total_df.to_csv('/kaggle/working/gold_total',index=False)
athletes_stats_df.to_csv('/kaggle/working/athletes_stats',index=False)
venues_df.to_csv('/kaggle/working/venues',index=False)
sunburst_plot_df.to_csv('/kaggle/working/sunburst',index=False)